In [1]:
__version__ = 1 + 1e-1 + 1j
__author__ = "Bavon C. K. Chao (赵庆华)"

from functools import partial
from json import dumps

import cv2
import numpy as np
from PIL import Image

In [2]:
# 将字典dump成json同时进行格式化
json_dumps = partial(dumps,
                     ensure_ascii=False,
                     indent=4,
                     separators=(', ', ': '))

In [3]:
SHARPEN_KERNEL = np.asarray(((0, -1, 0), (-1, 5, -1), (0, -1, 0)))
EMBOSS_KERNEL = np.asarray(((-2, -1, 0), (-1, 1, 1), (0, 1, 2)))
EDGE_SCHARR_KERNEL = np.asarray(
    ((-3 - 3j, 0 - 10j, 3 - 3j), (-10 + 0j, 0j, 10 + 0j), (-3 + 3j, 10j,
                                                           3 + 3j)))

In [4]:
import matplotlib.pyplot as plt


def fft(img):
    """傅利叶变换"""
    freq1 = np.fft.fft2(img)

    # NOTE np.vectorize 是构造numpy矩阵的一种方法
    print((freq1 == np.vectorize(complex)(freq1.real, freq1.imag)).all())
    freq2 = np.fft.fftshift(freq1)
    # freq2_low = np.copy(freq2)  # NOTE 用于低通滤波器运算
    (w, h) = freq2.shape[:2]
    w, h = map(lambda x: int(x / 2), (w, h))
    # 高通滤波，下面切片部分越大则影响越大
    freq2[w - 10:w + 11, h - 10:h + 11] = 0
    # # NOTE 低通滤波
    # freq2 -= freq2_low
    # FFT频谱图像
    spectrum_img = 20 * np.log10(np.abs(freq2 + 1e-2))
    # FFT相位图
    phase_img = np.angle(freq2)
    # 逆FFT重建图像；clip用于限制矩阵中的数的最大和最小值
    img_ = np.fft.ifft2(np.fft.ifftshift(freq2)).real
    # reduction_img = np.clip(img_, 0, 255)
    img_ -= np.min(img_)
    reduction_img = img_ / np.abs(np.max(img_)) * 255

    plt.subplot(2, 2, 1), plt.imshow(img,
                                     cmap="gray"), plt.title("Original Image",
                                                             size=20)
    plt.subplot(2, 2, 2), plt.imshow(spectrum_img, cmap="gray"), plt.title(
        "FFT Spectrum Magnitude", size=20)

    plt.subplot(2, 2, 3), plt.imshow(phase_img,
                                     cmap="gray"), plt.title("FFT Phase",
                                                             size=20)

    plt.subplot(2, 2,
                4), plt.imshow(reduction_img,
                               cmap="gray"), plt.title("Reconstructed Image",
                                                       size=20)

    plt.show()

In [5]:
def image_derivative(img):
    """计算图像导数"""
    kernel_x = np.asarray(((-1, 1), ))
    kernel_y = np.asarray(((-1, ), (1, )))
    img_x = cv2.filter2D(img, cv2.CV_8U, kernel_x)  # 横向偏导
    img_y = cv2.filter2D(img, cv2.CV_8U, kernel_y)  # 纵向偏导
    img_mag = np.sqrt(img_x**2 + pow(img_y, 2))  # 偏导图像合并
    img_dir = np.arctan(img_y / img_x)  # θ
    img_dir = np.where(np.isnan(img_dir), 0, img_dir)

In [6]:
def laplace(img):
    img_lplc = cv2.Laplacian(img, cv2.CV_8U, ksize=3)
    # 原始图像加拉普拉斯，得到锐化图像
    sharpened_img = np.clip(cv2.addWeighted(img, .8, img_lplc, .8, 0), 0, 255)
    display(Image.fromarray(sharpened_img))
    
    # img_ = img_lplc + img
    # img_ -= np.min(img_)
    # sharpened_img = img_ / np.abs(np.max(img_)) * 255
    # 反锐化
    img_delta = np.clip(img - img_lplc, 0, 255)
    sharpened_img = np.clip(img + img_delta * 5, 0, 255)
    display(Image.fromarray(sharpened_img))

In [7]:
def py_cpu_nms(dets, thresh):
    """非最大抑制算法，用于合并部分重合的候选框"""
    x1, y1, x2, y2, scores = dets[:, :5]
    
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return keep

In [8]:
img = cv2.imread("test.jpg", cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_eq = cv2.equalizeHist(img)  # 直方图均衡化，增加对比度
# CLAHE 有限对比适应性直方图均衡化
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
img_clahe = clahe.apply(img)

遍历不规则树状数据结构（比如cv2.findContours的返回值）伪代码
```python
contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE,
                                       cv2.CHAIN_APPROX_SIMPLE)
def parse_contours_tree(hierarchy, current_node):
    for node in current_node.sisters：
        # 处理具有同一个父节点的同级别所有节点
        if node.is_last_node:
            # 如果是最后一层节点，可能有特殊处理
        else:
            # 如果有子节点，处理当前节点，并处理子节点
            for sub_node in node.children:
                # 递归处理子节点的子节点
                parse_contours_tree(hierarchy, sub_node)
```

In [9]:
# 凸包算法，将点集合顺时针排序
points = cv2.convexHull(np.asarray(((1, 1), (1, 3), (3, 1), (3, 3))), clockwise=True)

# 判断一个点和一个点集合的位置关系，第三个参数表示是否计算点与点集合的距离
print(cv2.pointPolygonTest(points, (2.5, 2), False))
print(cv2.pointPolygonTest(points, (0, 2), False))
print(cv2.pointPolygonTest(points, (1, 2), False))

print(cv2.pointPolygonTest(points, (2.5, 2), True))
print(cv2.pointPolygonTest(points, (1, 2), True))
print(cv2.pointPolygonTest(points, (0, 2), True))

1.0
-1.0
0.0
0.5
-0.0
-1.0


np.arctan2 函数可以计算大于 180° 的角度